In [4]:
# I chose the Exponential Smoothing (Holt-Winters) model

# assignment1.py — Exponential Smoothing solution for the autograder

import os
import numpy as np
import pandas as pd
from statsmodels.tsa.holtwinters import ExponentialSmoothing

# ---------- data locations ----------
TRAIN_LOCAL = "assignment_data_train.csv"
TEST_LOCAL  = "assignment_data_test.csv"
TRAIN_URL   = "https://raw.githubusercontent.com/dustywhite7/econ8310-assignment1/main/assignment_data_train.csv"
TEST_URL    = "https://raw.githubusercontent.com/dustywhite7/econ8310-assignment1/main/assignment_data_test.csv"

def _read_csv_safe(local_path: str, fallback_url: str) -> pd.DataFrame:
    """
    Read from local file if it exists (this is how the grader runs).
    If not found (e.g., on Colab), fall back to the provided URL.
    """
    path = local_path if os.path.exists(local_path) else fallback_url
    return pd.read_csv(path, parse_dates=["Timestamp"])

# ---------- load & preprocess training ----------
train_df = _read_csv_safe(TRAIN_LOCAL, TRAIN_URL)
# Ensure chronological hourly index
train_df = train_df.sort_values("Timestamp").set_index("Timestamp").asfreq("H")

# Target cleaning: numeric + time interpolation to fill gaps
y_train = pd.to_numeric(train_df["trips"], errors="coerce")
y_train = y_train.interpolate(method="time", limit_direction="both")

# ---------- build (unfitted) model ----------
model = ExponentialSmoothing(
    y_train,
    trend="add",
    seasonal="add",
    seasonal_periods=24,
    initialization_method="estimated",
)

# ---------- fit ----------
modelFit = model.fit(optimized=True, use_brute=True)

# ---------- test horizon & forecast ----------
test_df = _read_csv_safe(TEST_LOCAL, TEST_URL)
h = len(test_df)                      # should be 744 hours for Jan 2019
forecast = modelFit.forecast(steps=h)

# ---------- predictions for the grader ----------
pred = np.asarray(forecast, dtype=float).ravel()
pred = np.maximum(pred, 0.0)          # avoid negatives


/tmp/ipython-input-3346509209.py:27: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  train_df = train_df.sort_values("Timestamp").set_index("Timestamp").asfreq("H")
